Run with the following conda environment: `../../conda_envs/training_env`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.pylab as pylab
import re

In [ ]:
params = {
    'legend.fontsize': 14,
    'legend.title_fontsize': 16,
    'axes.labelsize': 18,
    'axes.titlesize': 22,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'figure.titlesize': 25,
}
plt.rc('font', size=16)
plt.rc('pdf', fonttype=42)
pylab.rcParams.update(params)

In [ ]:
coda_name = 'DUMBO'
results_dir = '../../results/simu_output'
traces_dir = f'{results_dir}/../../traces'

In [ ]:
traces_l = []
hh_frac, trace_fmt, minutes = 0.02, f'{traces_dir}/caida/20160121-13{{}}00.UTC.anon.pcap', [46, 47, 48, 49, 50, 51, 52, 53, 54, 55]
traces_l.append((hh_frac, trace_fmt, minutes))
hh_frac, trace_fmt, minutes = 0.015, f'{traces_dir}/traces/mawi/{{}}.pcap', [1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925]
traces_l.append((hh_frac, trace_fmt, minutes))
hh_frac, trace_fmt, minutes = 0.03, f'{traces_dir}/traces/uni/{{}}.pcap', [146, 147, 148, 149, 151, 152, 153, 154, 155, 156]
traces_l.append((hh_frac, trace_fmt, minutes))

proto = 'all'
quantiles = ["0.75", "0.95"]

In [ ]:
avg_rel_error_model = [[] for _ in range(len(quantiles)*len(traces_l))]      
avg_rel_error_one_byte = [[] for _ in range(len(quantiles)*len(traces_l))]
avg_rel_error_two_bytes = [[] for _ in range(len(quantiles)*len(traces_l))]
avg_rel_error_doubled = [[] for _ in range(len(quantiles)*len(traces_l))]
avg_rel_error_pheavy = [[] for _ in range(len(quantiles)*len(traces_l))]
avg_rel_error_oracle = [[] for _ in range(len(quantiles)*len(traces_l))]

In [ ]:
for t_idx, (hh_frac, trace_fmt, minutes) in enumerate(traces_l):
    traces = [trace_fmt.format(mm) for mm in minutes]
    for trace in traces:
        trace_set, trace_name = trace.split('/')[-2:]
        trace_id = f"{trace_set}-{trace_name.split('.')[0]}"
        base_folder = f'{results_dir}/{trace_id}/{proto}/top_{hh_frac}_pct/iat'
        
        with open(f"{base_folder}/error/baseline.txt", "r") as f:
            output = f.read()
            for i, quantile in enumerate(quantiles):
                print(f"{quantile} {base_folder}/error/baseline.txt")
                baseline_1 = float(re.search(r'MRE {} 32 bins 1 byte:\s+([\d\.e-]+)'.format(quantile), output).group(1))
                baseline_2 = float(re.search(r'MRE {} 16 bins 2 bytes:\s+([\d\.e-]+)'.format(quantile), output).group(1))
                baseline_3 = float(re.search(r'MRE {} 32 bins 2 bytes:\s+([\d\.e-]+)'.format(quantile), output).group(1))
                avg_rel_error_one_byte[t_idx*len(quantiles)+i].append(baseline_1)
                avg_rel_error_two_bytes[t_idx*len(quantiles)+i].append(baseline_2)
                avg_rel_error_doubled[t_idx*len(quantiles)+i].append(baseline_3)

        with open(f"{base_folder}/error/oracle_531KB.txt", "r") as f:
            output = f.read()
            for i, quantile in enumerate(quantiles):
                print(f"{quantile} {base_folder}/error/oracle_531KB.txt")
                oracle = float(re.search(r'MRE {} Model:\s+([\d\.e-]+)'.format(quantile), output).group(1))
                avg_rel_error_oracle[t_idx*len(quantiles)+i].append(oracle)

        with open(f"{base_folder}/error/coda.txt", "r") as f:
            output = f.read()
            for i, quantile in enumerate(quantiles):
                print(f"{quantile} {base_folder}/error/coda.txt")
                coda = float(re.search(r'MRE {} Model:\s+([\d\.e-]+)'.format(quantile), output).group(1))
                avg_rel_error_model[t_idx*len(quantiles)+i].append(coda)

        with open(f"{base_folder}/error/pheavy.txt", "r") as f:
            output = f.read()
            for i, quantile in enumerate(quantiles):
                print(f"{quantile} {base_folder}/error/pheavy.txt")
                pheavy = float(re.search(r'MRE {} Model:\s+([\d\.e-]+)'.format(quantile), output).group(1))
                avg_rel_error_pheavy[t_idx*len(quantiles)+i].append(pheavy)

In [ ]:
# compute averages
for i, _ in enumerate(avg_rel_error_model):
    avg_rel_error_model[i] = np.median(avg_rel_error_model[i])
    avg_rel_error_one_byte[i] = np.median(avg_rel_error_one_byte[i])
    avg_rel_error_two_bytes[i] = np.median(avg_rel_error_two_bytes[i])
    avg_rel_error_doubled[i] = np.median(avg_rel_error_doubled[i])
    avg_rel_error_pheavy[i] = np.median(avg_rel_error_pheavy[i])
    avg_rel_error_oracle[i] = np.median(avg_rel_error_oracle[i])

In [ ]:
blue, orange, green, red, violet = plt.rcParams["axes.prop_cycle"].by_key()["color"][:5]

fig = plt.figure(figsize=(6,4.5))
gs = GridSpec(2, 3, height_ratios=[1, 3], width_ratios=[1, 1, 1], hspace=0.04)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)
ax3 = fig.add_subplot(gs[:, 1])
ax4 = fig.add_subplot(gs[:, 2])

bar_width = 0.17

r0 = np.arange(len(avg_rel_error_one_byte)) - bar_width*2
r1 = [x + 1*bar_width for x in r0]
r2 = [x + 2*bar_width for x in r0]
r3 = [x + 3*bar_width for x in r0]
r4 = [x + 4*bar_width for x in r0]

for ax in [ax1, ax2, ax3, ax4]:
    ax.bar(r0, avg_rel_error_two_bytes, label="16 bins, 2 bytes", width=bar_width, hatch="\\\\", edgecolor="black", color=blue)
    ax.bar(r1, avg_rel_error_one_byte, label="32 bins, 1 byte", width=bar_width, hatch="/", edgecolor="black", color=orange)
    ax.bar(r2, avg_rel_error_doubled, label="32 bins, 2 bytes\n(2x memory)", width=bar_width, hatch="--", edgecolor="black", color=violet)
    ax.bar(r3, avg_rel_error_pheavy, label="(w/ hints) pHeavy 5pk", width=bar_width, hatch="xx", edgecolor="black", color=red)    
    ax.bar(r4, avg_rel_error_model, label=f"(w/ hints) {coda_name}", width=bar_width, edgecolor="black", color=green)
    

if proto == 'all':
    ax1.set_ylim([0.20, 0.225])
    ax2.set_ylim([0.10, 0.145])
    ax3.set_ylim([0.013, 0.05])
    ax4.set_ylim([0.07, 0.22])
    ax1.set_yticks([0.21])
    ax2.set_yticks([0.11, 0.13])
    ax3.set_yticks([0.02, 0.03, 0.04, 0.05])
else:
    ax1.set_ylim([0.25, 0.28])
    ax2.set_ylim([0.125, 0.155])
    ax3.set_ylim([0.01, 0.05])
    ax1.set_yticks([0.27])
    ax2.set_yticks([0.13, 0.15])
    ax3.set_yticks([0.02, 0.03, 0.04, 0.05])

ax1.set_zorder(1)
ax2.set_zorder(1)
ax3.set_zorder(3)
plt.setp(ax1.yaxis.get_majorticklabels(), rotation=90, va='center')
plt.setp(ax2.yaxis.get_majorticklabels(), rotation=90, va='center')
plt.setp(ax3.yaxis.get_majorticklabels(), rotation=90, va='center')
plt.setp(ax4.yaxis.get_majorticklabels(), rotation=90, va='center')

ax1.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
ax1.tick_params(axis='x', which='both', top=False, bottom=False, labeltop=False, labelbottom=False)
ax2.tick_params(axis='x', which='both', top=False, labeltop=False)
ax3.tick_params(axis='x', which='both', top=False, labeltop=False)
ax4.tick_params(axis='x', which='both', top=False, labeltop=False)

d = .5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)

traces_labels = ['CAIDA', 'MAWI', 'UNI']

for ax in [ax1, ax2, ax3, ax4]:
    ax.set_xticks([i for i in range(len(quantiles)*len(traces_labels))])
    ax.set_xticklabels([f"{q[2:]}$^{{th}}$q." for trace in traces_labels for q in ['0.75', '0.95']], fontsize=14)

second_labels = traces_labels
second_ticks = [0.5, 2.5, 4.5]
for ax in [ax1, ax2, ax3, ax4]:
    ax.set_xticks(second_ticks, minor=True)
    ax.set_xticklabels(second_labels, minor=True)
    ax.tick_params(axis='x', which='major', pad=0)
    ax.tick_params(axis='x', which='minor', pad=25)
    ax.tick_params(axis='x', which='minor', length=0)
    ax.tick_params(axis='y', which='major', pad=-2)

ax1.set_xlim([-0.5, 1.5])
ax3.set_xlim([1.5, 3.5])
ax4.set_xlim([3.5, 5.5])

ax1.set_ylabel(None)
ax2.set_ylabel(None)
ax3.set_ylabel(None)
ax4.set_ylabel(None)
fig.text(0.07, 0.45, 'Mean Relative Error', va='center', ha='center', rotation='vertical', fontsize=plt.rcParams['axes.labelsize'])

box4 = ax4.get_position()
ax4.set_position([box4.x0, box4.y0, box4.width, box4.height * 0.64])
box3 = ax3.get_position()
ax3.set_position([box3.x0, box3.y0, box3.width, box3.height * 0.64])
box2 = ax2.get_position()
ax2.set_position([box2.x0, box2.y0, box2.width, box2.height * 0.8])
box1 = ax1.get_position()
ax1.set_position([box1.x0, box1.y0 - box2.height*0.2, box1.width, box1.height * 0.8])

# reorder legend
handles, labels = plt.gca().get_legend_handles_labels()
order = [0, 1, 2, 3, 4]
ax1.legend([handles[idx] for idx in order][3:], [labels[idx] for idx in order][3:], bbox_to_anchor=(-0.25, 0.9),
           loc='lower left', labelspacing=0.3, frameon=False, ncol=1, borderaxespad=0.05, handletextpad=0.5)
ax4.legend([handles[idx] for idx in order][:3], [labels[idx] for idx in order][:3], bbox_to_anchor=(1.0, 1.),
           loc='lower right', labelspacing=0.3, frameon=False, ncol=1, borderaxespad=0.05, handletextpad=0.5)

plt.savefig(f'out/iat_q{"-".join(quantiles)}.stacked.{proto}.w-uni.pdf', format='pdf', dpi=1200, bbox_inches="tight")
plt.show()